In [1]:
%load_ext autoreload
%autoreload 2

In [195]:
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn import datasets
from xgboost import XGBClassifier
from sklearn.metrics import auc, accuracy_score, roc_auc_score
import pickle
from sklearn import metrics

import pandas as pd
import numpy as np
import pickle
import json

from helpers.utils import load_data

In [2]:
X = np.load("data/MaskDataCorrected.npz")['X'][()]
Y = np.load("data/MaskDataCorrected.npz")['y'][()]
test_X = X[0:50000, :]
train_X = X[50000:, :]
train_Y = Y[50000:]

In [3]:
X.shape

(1578627, 1000)

In [12]:
X = np.load("data/bagOfWods_5000.npz")['X'][()]
Y = np.load("data/bagOfWods_5000.npz")['y'][()]
test_X = X[0:50000, :]
train_X = X[50000:, :]
train_Y = Y[50000:]

In [13]:
X.shape

(1578627, 5000)

array([-1, -1, -1, ..., -1, -1,  0])

In [583]:
X_train, X_test, y_train, y_test = train_test_split(train_X[:2000, :150], train_Y[:2000], test_size= 0.2, random_state= 0)

In [584]:
X_train.shape
y_train.shape
X_test.shape

(400, 150)

In [585]:
dtrain = xgboost.DMatrix(X_train, label=y_train)
num_round = 1000
nfold = 15

In [586]:
param = {'bst:max_depth':11, 'bst:eta':0.05, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 8
param['eval_metric'] = 'error'

In [587]:
result = xgboost.cv(param, dtrain, num_round, nfold=nfold,seed = 0)

In [588]:
result.head(3)

test-error-mean  test-error-std  train-error-mean  train-error-std
0         0.376729        0.052296          0.272013         0.009617
1         0.375472        0.054814          0.263702         0.008966
2         0.377987        0.052068          0.258715         0.007404

In [589]:
idx = np.asscalar(np.argwhere(result["test-error-mean"] == np.min(result["test-error-mean"]))[0])
idx

180

In [603]:
xgb_params = {'max_depth' : [11], 'learning_rate':[0.05],
                 'n_estimators': [idx + 100],
                  "objective": ["binary:logistic"], 'nthread' : [36]}
xgb_params

{'learning_rate': [0.05],
 'max_depth': [11],
 'n_estimators': [280],
 'nthread': [36],
 'objective': ['binary:logistic']}

In [604]:
xgb = XGBClassifier()
grid = GridSearchCV(xgb, xgb_params, scoring='accuracy')

In [605]:
grid.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [280], 'objective': ['binary:logistic'], 'learning_rate': [0.05], 'max_depth': [11], 'nthread': [36]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [606]:
result0 = grid.predict(X_test)

In [607]:
accuracy_score(y_test, result0)

0.63500000000000001

In [311]:
model = xgboost.train(param, dtrain, 150)
model

In [312]:
pred = model.predict(xgboost.DMatrix(X_test))

fpr, tpr, thresholds = metrics.roc_curve(y_test, pred, pos_label=1)

In [313]:
metrics.auc(fpr, tpr)

0.75578947368421046

In [314]:
result = [1 if item >= 0.50 else 0 for item in pred]

In [315]:
accuracy = accuracy_score(y_test, result)
accuracy

0.69499999999999995

In [79]:
np.max(result["test-auc-mean"])

0.67556050000000001

In [20]:
xgb_params = {'max_depth' : 11, "objective": "binary:logistic",
                       'subsample' : 0.05, 'colsample_bytree': 0.6,
                       'n_estimators' : 1000,'learning_rate' : 0.1, 'gamma':0}

In [21]:
xgboost.cv(xgb_params)

TypeError: cv() takes at least 2 arguments (1 given)

In [139]:
xgb_params = {'max_depth' : [11], "objective": ["binary:logistic"],
                       'subsample' : [0.05, 0.1], 'colsample_bytree': [0.6],
                       'n_estimators' : [1000],'learning_rate' : [0.05, 0.1], 'gamma':[0]}
xgb = XGBClassifier()
grid = GridSearchCV(xgb, xgb_params, scoring='roc_auc', cv = 15, n_jobs=-1)

In [140]:
grid.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [238]:
grid.fit(X_train, y_train)

GridSearchCV(cv=15, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'colsample_bytree': [0.6], 'learning_rate': [0.05, 0.1], 'n_estimators': [1000], 'subsample': [0.05, 0.1], 'objective': ['binary:logistic'], 'max_depth': [11], 'gamma': [0]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [243]:
output = open("cache/xgboost_grid.pkl", 'wb')
pickle.dump(grid, output)
output.close()

In [244]:
# pkl = open("model.pkl", 'rb')
# grid2 = pickle.load(pkl)
# pkl.close()

In [247]:
best_estimator = grid2.best_estimator_
best_estimator

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.6,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=11,
       min_child_weight=1, missing=nan, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.1)

In [248]:
score = grid.best_score_
score

0.60125623219373225

In [249]:
best_params = grid.best_params_

In [234]:
result = grid.best_estimator_.predict(X_test)

In [250]:
accuracy = accuracy_score(y_test, result)

In [252]:
report = {
    "best_estimator" : predictions,
    "best_score" : score,
    "best_params" : best_params,
    "accuracy" : accuracy
}

In [256]:
report_str = str(report)

In [258]:
with open("report/report.json", "w") as outfile:
    json.dump(report_str, outfile)

In [262]:
test_result = grid.best_estimator_.predict(test_X)

In [263]:
submission = pd.DataFrame({'id': np.arange(1, 50001), 'y': test_result})
submission.to_csv("submission/filename.csv", index = False)

(50000,)

In [266]:
np.arange(1, 50000 + 1)

array([    1,     2,     3, ..., 49998, 49999, 50000])

In [18]:
models = {
#     'RandomForestClassifier' : RandomForestClassifier(),
#     'GradientBoostingClassifier' : GradientBoostingClassifier(),
    'XGBClassifier' : XGBClassifier()

}

In [19]:
random_state = 30

In [38]:
#     'RandomForestClassifier' : {'n_estimators': [500, 600], 'criterion': ['gini', 'entropy'], 'max_depth' : [15, 20, 25]},
#     'ExtraTreesClassifier' : {'n_estimators': [500, 600], 'criterion': ['gini', 'entropy'], 'max_depth' : [15, 20, 25]},
#     'AdaBoostClassifier' : {'n_estimator': [1000, 2000], 'learning_rate': [0.1, 0.2]},
xgb_params = {'max_depth' : [11], "objective": ["binary:logistic"],
                       'subsample' : [0.05, 0.1], 'colsample_bytree': [0.6],
                       'n_estimators' : [1000],'learning_rate' : [0.1, 0.2], 'gamma':[0]}
#     ,
#     'GradientBoostingClassifier' : {'max_depth' : [7, 10],
#                        'n_estimators' : [1000, 2000],'learning_rate' : [0.1, 0.2]},
#     'SVC' : {'kernel':('linear', 'rbf'), 'C':[1, 5, 10]}

Fitting 10 folds for each of 4 candidates, totalling 40 fits
[CV] colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0 
[CV] colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0 
[CV] colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0 
[CV] colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0 
[CV] colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0 
[CV] colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0 
[CV] colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0 
[CV] colsample_bytree=

[Parallel(n_jobs=-1)]: Batch computation too fast (0.1980s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s


[CV]  colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0, score=0.614898 -   0.2s
[CV]  colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0, score=0.642245 -   0.2s
[CV] colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.1, objective=binary:logistic, max_depth=11, gamma=0 
[CV] colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.1, objective=binary:logistic, max_depth=11, gamma=0 
[CV]  colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.1, objective=binary:logistic, max_depth=11, gamma=0, score=0.611765 -   0.2s
[CV]  colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.1, objective=binary:logistic, max_depth=11, gamma=0, score=0.604118 -   0.2s
[CV] colsample_bytree=0.6, learning_rate=0.2, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.4s


[CV]  colsample_bytree=0.6, learning_rate=0.2, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0, score=0.491373 -   0.2s
[CV]  colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.1, objective=binary:logistic, max_depth=11, gamma=0, score=0.597347 -   0.2s
[CV]  colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.1, objective=binary:logistic, max_depth=11, gamma=0, score=0.644400 -   0.2s
[CV] colsample_bytree=0.6, learning_rate=0.2, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0 
[CV] colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.1, objective=binary:logistic, max_depth=11, gamma=0 
[CV] colsample_bytree=0.6, learning_rate=0.1, n_estimators=1000, subsample=0.1, objective=binary:logistic, max_depth=11, gamma=0 
[CV]  colsample_bytree=0.6, learning_rate=0.2, n_estimators=1000, subsample=0.05, objective=binary:logistic, max_depth=11, gamma=0, score=0.654

[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    1.1s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'colsample_bytree': [0.6], 'learning_rate': [0.1, 0.2], 'n_estimators': [1000], 'subsample': [0.05, 0.1], 'objective': ['binary:logistic'], 'max_depth': [11], 'gamma': [0]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=10)

In [56]:
print(predictions)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.6,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=11,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.1)


In [58]:
print(score)

0.61056819928


In [72]:
try:
    with open("saved_params.json") as f:
        saved_params = json.load(f)
except IOError:
    saved_params = {}

In [24]:
for model_name, model in models.items():
    
    print("running %s" %model_name)
    clf = GridSearchCV(model, PARAM_GRID[model_name], cv=12, n_jobs=-1, scoring="roc_auc")
    pred = clf.fit(x, y)
    with open("cache/models/%s.pkl" %model_name, 'wb') as f:
        pickle.dump(pred, f, pickle.HIGHEST_PROTOCOL)
        
    saved_params[model_name] = {
        "score" : clf.best_score_,
        "params" : clf.best_params_
    }
    print(clf.best_score_)

running XGBClassifier


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code

In [74]:
with open("saved_params.json", 'w') as f: json.dump(saved_params, f, indent=4)

In [364]:
with open('report/report0.json') as f:
    data = json.load(f)

In [368]:
with open("report/report0.json", mode='w') as f:
    json.dump([], f)

In [369]:
with open("report/report0.json", mode='r') as feedsjson:
        feeds = json.load(feedsjson)

In [374]:
with open("report/report0.json", mode='w') as feedsjson:
    entry = {'name': "args.name", 'url': "args.url"}
    feeds.append(entry)
    json.dump(feeds, feedsjson)

In [366]:
a_dict = {'name' : 'minsu'}
data.update(str(a_dict))

with open('report/report0.json', 'w') as f:
    json.dump(data, f)

AttributeError: 'unicode' object has no attribute 'update'

In [379]:
from time import gmtime, strftime
import time

In [386]:
int(time.time())

1461793411

In [388]:
"good %s" %10

'good 10'

In [391]:
name_report = "report_%s.json" %int(time.time())

In [392]:
name_report

'report_1461793508.json'

In [406]:
"nice %s %s %s" %("yo", 'ho', 11)

'nice yo ho 11'

In [608]:
model_dic = {

    "randomForest" : RandomForestClassifier(n_jobs=-1, n_estimators=1000, max_depth=10)

}

In [613]:
for key, item in model_dic.items():
    print(key)
    print(item)

randomForest
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
